In [1]:
import pandas as pd
import numpy as np

In [2]:
#load data

In [3]:
daily_df = pd.read_csv("dataset/stations_daily.csv")

In [4]:
hourly_df = pd.read_csv("dataset/stations_hourly.csv")

In [5]:
rsinaica_df = pd.read_csv("dataset/stations_rsinaica.csv")

# จัดการกับข้อมูลรายงานฝุ่นรายวันก่อน

we will use this dataframe to calculate the yearly statistic of ฝุ่น

In [6]:
daily_df.head()

,datetime,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,...,HCT,HRI,IUV,PB,PP,PST,RS,TMPI,UVA,XIL
0,2015-04-02,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.361000,NaN,NaN
1,2015-05-21,32,0.454967,0.347400,0.059194,0.029176,NaN,44.879091,0.029525,0.053121,...,NaN,NaN,NaN,570.775455,0.296364,NaN,7.871636,16.792818,NaN,NaN
2,2015-05-22,32,0.077821,0.389323,0.019158,0.051903,NaN,42.506000,0.009589,0.014215,...,NaN,NaN,NaN,558.360000,0.002750,NaN,384.788079,17.018949,NaN,NaN
3,2015-05-23,32,0.018036,0.387729,0.019711,0.021092,0.144603,54.734437,0.009863,0.013436,...,NaN,NaN,NaN,573.282187,0.038750,NaN,252.605975,18.575000,NaN,NaN
4,2015-05-24,32,0.024935,0.398042,0.024757,0.009568,0.494143,58.374574,0.012397,0.010337,...,NaN,NaN,NaN,572.169574,0.003404,NaN,259.231664,16.781064,NaN,NaN


In [7]:
daily_df.shape

(231592, 27)

In [8]:
daily_df.head()

,datetime,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,...,HCT,HRI,IUV,PB,PP,PST,RS,TMPI,UVA,XIL
0,2015-04-02,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.361000,NaN,NaN
1,2015-05-21,32,0.454967,0.347400,0.059194,0.029176,NaN,44.879091,0.029525,0.053121,...,NaN,NaN,NaN,570.775455,0.296364,NaN,7.871636,16.792818,NaN,NaN
2,2015-05-22,32,0.077821,0.389323,0.019158,0.051903,NaN,42.506000,0.009589,0.014215,...,NaN,NaN,NaN,558.360000,0.002750,NaN,384.788079,17.018949,NaN,NaN
3,2015-05-23,32,0.018036,0.387729,0.019711,0.021092,0.144603,54.734437,0.009863,0.013436,...,NaN,NaN,NaN,573.282187,0.038750,NaN,252.605975,18.575000,NaN,NaN
4,2015-05-24,32,0.024935,0.398042,0.024757,0.009568,0.494143,58.374574,0.012397,0.010337,...,NaN,NaN,NaN,572.169574,0.003404,NaN,259.231664,16.781064,NaN,NaN


Let's add year and month columns to daily_df

In [9]:
daily_df.insert(1, "year", daily_df["datetime"].apply(lambda x: x.split("-")[0]))
daily_df.insert(1, "month", daily_df["datetime"].apply(lambda x: x.split("-")[1]))

although month and year is not an int yet we will convert it to int first

In [17]:
daily_df["month"] = pd.to_numeric(daily_df["month"])
daily_df["year"] = pd.to_numeric(daily_df["year"])
daily_df

,datetime,month,year,station_id,PM2.5,PM10,NOx,O3,CO,HR,...,HCT,HRI,IUV,PB,PP,PST,RS,TMPI,UVA,XIL
0,2015-04-02,4,2015,32,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.361000,NaN,NaN
1,2015-05-21,5,2015,32,0.454967,0.347400,0.059194,0.029176,NaN,44.879091,...,NaN,NaN,NaN,570.775455,0.296364,NaN,7.871636,16.792818,NaN,NaN
2,2015-05-22,5,2015,32,0.077821,0.389323,0.019158,0.051903,NaN,42.506000,...,NaN,NaN,NaN,558.360000,0.002750,NaN,384.788079,17.018949,NaN,NaN
3,2015-05-23,5,2015,32,0.018036,0.387729,0.019711,0.021092,0.144603,54.734437,...,NaN,NaN,NaN,573.282187,0.038750,NaN,252.605975,18.575000,NaN,NaN
4,2015-05-24,5,2015,32,0.024935,0.398042,0.024757,0.009568,0.494143,58.374574,...,NaN,NaN,NaN,572.169574,0.003404,NaN,259.231664,16.781064,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231587,2021-12-27,12,2021,426,28.956522,67.375000,0.036429,0.030750,1.684583,56.541667,...,NaN,NaN,NaN,708.600000,0.000000,NaN,136.416667,NaN,NaN,NaN
231588,2021-12-28,12,2021,426,20.818182,55.833333,0.037508,0.026792,1.700000,44.375000,...,NaN,NaN,NaN,705.670833,0.000000,NaN,136.875000,NaN,NaN,NaN
231589,2021-12-29,12,2021,426,16.857143,51.458333,0.045588,0.017083,1.732917,37.875000,...,NaN,NaN,NaN,707.687500,0.000000,NaN,108.708333,NaN,NaN,NaN
231590,2021-12-30,12,2021,426,21.555556,51.750000,0.037229,0.029833,1.520000,42.916667,...,NaN,NaN,NaN,710.141667,0.000000,NaN,129.666667,NaN,NaN,NaN


Now we will manage Nan value

sort values by year and month

In [18]:
(daily_df.isna().sum() * 100) / len(daily_df) # shocked

datetime        0.000000
month           0.000000
year            0.000000
station_id      0.000000
PM2.5          30.179799
PM10           22.069415
NOx            51.258679
O3             16.682355
CO             24.223203
HR             46.766296
NO             48.981830
NO2            26.903347
TMP            45.155705
BEN            99.665360
CH4            99.883416
CN             99.743083
CO2            99.975820
H2S            99.879098
HCNM           99.879530
HCT           100.000000
HRI            95.593544
IUV            97.868666
PB             60.388096
PP             67.466061
PST           100.000000
RS             67.396542
TMPI           85.152769
UVA            93.052437
XIL            99.665360
dtype: float64

# hourly_df

In [10]:
hourly_df.head()

,datetime,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,...,HCT,HRI,IUV,PB,PP,PST,RS,TMPI,UVA,XIL
0,2015-04-02 09:00:00,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.361,NaN,NaN
1,2015-05-21 18:00:00,32,0.97112,NaN,NaN,NaN,NaN,24.416,NaN,NaN,...,NaN,NaN,NaN,569.61,0.00,NaN,41.1090,16.723,NaN,NaN
2,2015-05-21 19:00:00,32,0.97122,0.18008,0.111060,0.020706,NaN,31.282,0.005918,0.104480,...,NaN,NaN,NaN,570.01,1.55,NaN,8.0007,16.745,NaN,NaN
3,2015-05-21 19:00:00,32,0.97122,0.18008,0.111060,0.020706,NaN,31.282,0.104480,0.104480,...,NaN,NaN,NaN,570.01,1.55,NaN,8.0007,16.745,NaN,NaN
4,2015-05-21 20:00:00,32,0.93155,0.38415,0.058019,0.030228,NaN,36.145,0.003471,0.054585,...,NaN,NaN,NaN,569.97,0.00,NaN,7.3412,16.586,NaN,NaN


In [11]:
hourly_df.groupby("station_id").describe()["PM2.5"]

,count,mean,std,min,25%,50%,75%,max
station_id,,,,,,,,
32,27181.0,1.383881e+02,3.250771e+02,0.0,0.022950,0.049291,26.1610,2.052900e+03
33,38999.0,1.080567e+02,3.808642e+02,0.0,0.297935,6.841800,18.6150,3.928400e+04
53,42037.0,2.575601e+01,8.195365e+01,0.0,9.800000,17.900000,24.3000,9.850000e+02
56,31985.0,1.966463e+01,1.910038e+01,0.0,6.661100,16.000000,28.3940,5.395100e+02
59,49440.0,1.290825e+35,7.561844e+36,0.0,9.298075,16.245500,26.9215,4.924800e+38
...,...,...,...,...,...,...,...,...
359,34390.0,1.196672e+01,7.384815e+00,1.0,7.101695,11.000000,15.0000,3.705254e+02
388,24775.0,3.208876e+01,1.130053e+02,0.0,10.000000,16.000000,24.0000,9.860000e+02
424,28556.0,2.040129e+01,1.608435e+01,0.0,10.000000,17.000000,27.0000,4.860000e+02


In [12]:
rsinaica_df.head()

,station_id,station_name,station_code,network_id,network_name,network_code,street,ext,interior,colonia,...,date_validated2,passed_validation,video,lat,lon,date_started,timezone,street_view,video_interior,color
0,397,"CENICA 1, UAM Iztapalapa",CEN,144,NaN,ESTSP,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,19.489583,-98.801083,2002-01-01,"Tiempo del centro, UTC-6 (UTC-5 en verano)",NaN,NaN,Others
1,344,C. Salud Tlaxcoapán,TCP,127,NaN,TCP,NaN,NaN,NaN,NaN,...,NaN,NaN,Array,20.091472,-99.228417,NaN,"Tiempo del centro, UTC-6 (UTC-5 en verano)",NaN,NaN,Others
2,365,Costa Baja,UM,140,NaN,LAP,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2010-03-14,"Tiempo del centro, UTC-6 (UTC-5 en verano)",NaN,NaN,Others
3,393,Unidad Móvil de Hidalgo,UMHI,143,NaN,POP,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,19.489583,-98.801083,2013-05-24,"Tiempo del centro, UTC-6 (UTC-5 en verano)",NaN,NaN,Others
4,394,Unidad Móvil Iztapalapa,UMIZ,143,NaN,POP,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,19.489583,-98.801083,2013-05-21,"Tiempo del centro, UTC-6 (UTC-5 en verano)",NaN,NaN,Others


In [13]:
rsinaica_df.sort_values("station_id")

,station_id,station_name,station_code,network_id,network_name,network_code,street,ext,interior,colonia,...,date_validated2,passed_validation,video,lat,lon,date_started,timezone,street_view,video_interior,color
14,30,Secundaria Leyes de Reforma,LEY,30,Aguascalientes,AGS,Av. Pozo Bravo,s/n,s/n,Fracc. Revolución,...,2013-12-31,0.0,NaN,21.903000,-102.284611,2012-12-01,"Tiempo del centro, UTC-6 (UTC-5 en verano)",NaN,NaN,Others
10,31,CBTIS,CBT,30,Aguascalientes,AGS,Niza,S/N,NaN,Del Valle Segunda Sección,...,NaN,NaN,https://www.youtube.com/embed/SNcAjJB_zi8,21.873311,-102.320803,2010-08-24,"Tiempo del centro, UTC-6 (UTC-5 en verano)","https://www.google.com.mx/maps/@21.8727525,-10...",https://www.youtube.com/embed/tggsv9S0lDM,Others
17,32,Secretaría de Medio Ambiente,SMA,30,Aguascalientes,AGS,"Blvd. José María Chávez, esq. Av. Siglo XXI",No. 1917,s/n,Prados de Villasunción,...,NaN,NaN,https://www.youtube.com/embed/wQxHKOIv0Qw,21.846392,-102.288431,2015-01-16,"Tiempo del centro, UTC-6 (UTC-5 en verano)","https://www.google.com.mx/maps/@21.845408,-102...",https://www.youtube.com/embed/wE0AYV2dHWo,Others
11,33,Centro,CEN,30,Aguascalientes,AGS,Rivero y Gutiérrez esquina con Morelos,s/n,s/n,Zona Centro,...,NaN,NaN,https://www.youtube.com/embed/MfwTsxxPEic,21.883781,-102.295825,2016-01-01,"Tiempo del centro, UTC-6 (UTC-5 en verano)","https://www.google.com.mx/maps/@21.8836364,-10...",https://www.youtube.com/embed/QPYSASjQJKw,Others
13,34,ISSSTE,ISSSTE,30,Aguascalientes,AGS,Av. Universidad,s/n,s/n,Fraccionamiento San Cayetano,...,NaN,NaN,NaN,21.871139,-102.293972,NaN,"Tiempo del centro, UTC-6 (UTC-5 en verano)","https://www.google.com.mx/maps/@21.902442,-102...",NaN,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,422,Instituto Tecnológico del Centla,ITCE,148,Centla,CTA,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,18.541050,-92.650142,NaN,"Tiempo del centro, UTC-6 (UTC-5 en verano)",NaN,NaN,Others
191,423,Purísima del Rincón,PUR,149,Purísima del Rincón,PUR,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2017-06-07,Tiempo del centro (UTC-6 todo el año),NaN,NaN,Others
140,424,Cadereyta,SE3,72,Monterrey,MTY,"Campesina cruz con Vicente Guerrero,",NaN,NaN,Jerónimo Treviño 2do. Sector,...,NaN,NaN,NaN,25.600556,-99.994167,2018-02-06,"Tiempo del centro, UTC-6 (UTC-5 en verano)",NaN,NaN,Others
133,425,Universidad,N2,72,Monterrey,MTY,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,25.730000,-100.309722,2018-01-29,"Tiempo del centro, UTC-6 (UTC-5 en verano)",NaN,NaN,Others
